In [24]:
import pandas as pd
import collections
from sklearn.ensemble import RandomForestClassifier

import re
import numpy as np
from datetime import datetime 
from nltk.tokenize import word_tokenize
import data_io,SIF_embedding
import scipy
import kmeans
import natural_language_processing

In [3]:
train=pd.read_json("train_original_tweets_top1000full_text_only.json")
test=pd.read_json("test_original_tweets_top1000full_text_only.json")

                created_at                                           entities  \
0      2018-04-07 04:06:52  {'urls': [{'url': 'https://t.co/s0xEcWFiqn', '...   
1      2018-04-07 03:54:20  {'urls': [{'url': 'https://t.co/Dqgw5v9a13', '...   
10     2018-04-03 04:02:16  {'urls': [], 'hashtags': [{'text': 'americanid...   
100    2018-03-20 04:14:46  {'urls': [], 'hashtags': [], 'user_mentions': []}   
1000   2018-03-05 02:35:43  {'urls': [], 'hashtags': [], 'user_mentions': []}   
10000  2018-03-23 01:10:56  {'urls': [{'url': 'https://t.co/Ma4hOcN0TO', '...   
100000 2018-03-09 15:32:47  {'urls': [{'url': 'https://t.co/ASleT4U6tP', '...   
100001 2018-03-09 15:28:54  {'urls': [], 'hashtags': [], 'user_mentions': ...   
100002 2018-03-09 15:12:49  {'urls': [{'url': 'https://t.co/ASleT4U6tP', '...   
100003 2018-03-09 15:02:37  {'urls': [{'url': 'https://t.co/rn1fmdQFY7', '...   
100004 2018-03-09 14:27:59  {'urls': [{'url': 'https://t.co/ASleT5bHSp', '...   
100005 2018-03-09 14:14:10  

In [11]:
start_time = "Wed Apr 15 00:00:00 +0000 2018"
start_time = datetime.strptime(start_time, "%a %b %d %H:%M:%S +0000 %Y")

test=test[test["created_at"]<start_time]=test[test["created_at"]<start_time]

In [12]:
print(len(train))
print(len(test))
print(len(train)/(len(test)+len(train)))

118604
20157
0.8547358407621738


In [31]:
def cleanup_text(texts):
    cleaned_text = []
    for text in texts:
        # remove &quot and &amp
        text = re.sub(r'&quot;(.*?)&quot;', "\g<1>", text)
        text = re.sub(r'&amp;', "", text)

        # replace emoticon
        text = re.sub(r'(^| )(\:\w+\:|\<[\/\\]?3|[\(\)\\\D|\*\$][\-\^]?[\:\;\=]|[\:\;\=B8][\-\^]?[3DOPp\@\$\*\\\)\(\/\|])(?=\s|[\!\.\?]|$)', "\g<1>TOKEMOTICON", text)

        text = text.lower()
        text = text.replace("tokemoticon", "TOKEMOTICON")

        # replace url
        text = re.sub(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?',
                    "TOKURL", text)

        # replace mention
        text = re.sub(r'@[\w]+', "TOKMENTION", text)

        # replace hashtag
        text = re.sub(r'#[\w]+', "TOKHASHTAG", text)

        # replace dollar
        text = re.sub(r'\$\d+', "TOKDOLLAR", text)

        
        # remove multiple spaces
        text = re.sub(r' +', ' ', text)

        # remove newline
        text = re.sub(r'\n', ' ', text)

        cleaned_text.append(text)
    return cleaned_text


def get_average_word_length(words,english_punctuations):
    word_count=0
    word_total_length=0
    for word in words:
        if word not in english_punctuations:
            word_total_length+=len(word)
            word_count+=1
    return word_total_length/word_count

def generate_common_features(train_df):
    english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
    train_df["tokenized_words"]=train_df["text"].apply(lambda x:word_tokenize(x))
    train_df["hashtag_num"]=train_df["entities"].apply(lambda x:len(x["hashtags"]))
    train_df["link_num"]=train_df["entities"].apply(lambda x:len(x["urls"]))
    train_df["mention_num"]=train_df["entities"].apply(lambda x:len(x["user_mentions"]))
    train_df["time_of_date"]=train_df["created_at"].apply(lambda x:x.hour*24+x.minute)
    train_df["len_of_tweet"]=train_df["text"].apply(lambda x:len(x))
    train_df["avg_len_of_word"]=train_df["tokenized_words"].apply(lambda x:get_average_word_length(x,english_punctuations))
    train_df["word_count"]=train_df["tokenized_words"].apply(lambda x:len(x))
    
    
#4 features involving most tweetable words
most_tweetable_words={"you":0,"twitter":1,"please":2,"retweet":3,"post":4,"blog":5,"social":6,
                     "free":7,"media":8,"help":9,"great":10,"10":11,"follow":12,"how":13,
                      "to":14,"top":15,"check":16,"out":17,"this":18,"people":19,"vote":20,
                     "?":21}
def add_most_tweetable_words_features(pd):
    pd["aggreg_exist"]=[1 if any(t in most_tweetable_words for t in i.split()) else 0 for i in pd["text"]]
    pd["aggreg_tf"]=[sum([1 if t in most_tweetable_words else 0 for t in i.split()]) for i in pd["text"]]
    for i in most_tweetable_words:
        pd[i+"_exist"]=[1 if any(t==i for t in j.split()) else 0 for j in pd["text"]]
        pd[i+"_tf"]=[sum([1 if t==i else 0 for t in j.split()]) for j in pd["text"]]

def add_question_features(pd):
    pd["ques_exist"]=[1 if collections.Counter(i)["?"]!=0 else 0 for i in pd["text"]]
    pd["ques_exist"]=[collections.Counter(i)["?"] for i in pd["text"]]

def add_pn_word_features(pd):
    pd["positive_word"]=[sum([1 if t in positive_words else 0 for t in j.split()]) for j in pd["text"]]
    pd["negative_word"]=[sum([1 if t in negative_words else 0 for t in j.split()]) for j in pd["text"]]
    

f=open("positive_words.txt")
positive_words=set()
for line in f.readlines():
    positive_words.add(line.strip())
f.close()

f=open("negative_words.txt")
negative_words=set()
for line in f.readlines():
    negative_words.add(line.strip())
f.close()

In [17]:
train["text"]=cleanup_text(train["text"].values)
test["text"]=cleanup_text(test["text"].values)

In [10]:
wordfile = 'glove.twitter.27B/glove.twitter.27B.100d.txt' # word vector file, can be downloaded from GloVe website
weightfile = 'enwiki_vocab_min200.txt' # each line is a word and its frequency
weightpara = 1e-3 # the parameter in the SIF weighting scheme, usually in the range [3e-5, 3e-3]
rmpc = 1 # number of principal components to remove in SIF weighting scheme
# load word vectors
(words, We) = data_io.getWordmap(wordfile)
# load word weights
word2weight = data_io.getWordWeight(weightfile, weightpara) # word2weight['str'] is the weight for the word 'str'
weight4ind = data_io.getWeight(words, word2weight) # weight4ind[i] is the weight for the i-th word
# load sentences

In [9]:
def generate_SIF_embedding_vectors(df,weight4ind,words,We,rmpc):
    x, m = data_io.sentences2idx(df["text"].values, words) # x is the array of word indices, m is the binary mask indicating whether there is a word in that location
    w = data_io.seq2weight(x, m, weight4ind) # get word weights
    embedding = SIF_embedding.SIF_embedding(We, x, w, rmpc)
    embed_list=list(map(lambda x:x,embedding))
    return embed_list

In [18]:
train["embed"]=generate_SIF_embedding_vectors(train,weight4ind,words,We,rmpc)
test["embed"]=generate_SIF_embedding_vectors(test,weight4ind,words,We,rmpc)

In [19]:
train["greater_times"]=train["retweet_count"]/train["retweet_median"]
popular_train_df=train[train["greater_times"]>=10]
unpopular_train_df=train[train["greater_times"]<=0.5]
hot_vectors=np.array([x for x in popular_train_df["embed"].values])
cold_vectors=np.array([x for x in unpopular_train_df["embed"].values])

In [28]:
k = 5
KM = kmeans.KMeans()
y,centers_pop=KM.train(hot_vectors,KM.init_centers(hot_vectors,k),niters=300)
KM = kmeans.KMeans()
y,centers_unpop=KM.train(cold_vectors,KM.init_centers(cold_vectors,k),niters=300)

In [29]:
for i in range(5):    
    train["pop_dist_"+str(i)]=train["embed"].apply(lambda x:np.linalg.norm(x-centers_pop[i]))
    train["uppop_dist_"+str(i)]=train["embed"].apply(lambda x:np.linalg.norm(x-centers_unpop[i]))
    test["pop_dist_"+str(i)]=test["embed"].apply(lambda x:np.linalg.norm(x-centers_pop[i]))
    test["uppop_dist_"+str(i)]=test["embed"].apply(lambda x:np.linalg.norm(x-centers_unpop[i]))
    

In [32]:
generate_common_features(train)
generate_common_features(test)

add_most_tweetable_words_features(train)
add_question_features(train)
add_most_tweetable_words_features(test)
add_question_features(test)
add_pn_word_features(train)
add_pn_word_features(test)

In [36]:
train=train[train["word_count"]>=3]
test=test[test["word_count"]>=3]

In [37]:
lang_model = natural_language_processing.LanguageModel(popular_train_df["text"].values, 3)
train["perplexity"]=train["text"].apply(
    lambda x:lang_model.perplexity(x))

test["perplexity"]=test["text"].apply(
    lambda x:lang_model.perplexity(x))


/Users/luxiaowei/anaconda2/envs/python35/lib/python3.5/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [45]:
train_data=train.copy()
test_data=test.copy()

In [46]:
# train_target=train_data["retweet_median"]
train_data["diff"]=train_data["retweet_count"]/train_data["retweet_median"]
train_data["target"]=[1 if i>=2 else 0 for i in train_data["diff"]]

test_data["diff"]=test_data["retweet_count"]/test_data["retweet_median"]
test_data["target"]=[1 if i>=2 else 0 for i in test_data["diff"]]

In [39]:
feature_columns=['perplexity','word_count', 'hashtag_num', 'link_num',
       'mention_num', 'time_of_date', 'len_of_tweet', 'avg_len_of_word',
        'twitter_tf','this_tf', 'to_tf','how_tf','great_tf', 'check_tf',
       'retweet_tf', 'free_tf','blog_tf', '?_tf', 'help_tf', 'post_tf',
        'top_tf','please_tf','out_tf', 'social_tf','10_tf',
        'follow_tf', 'you_tf',
       'vote_tf','people_tf', 'media_tf', 'pop_dist_0', 'uppop_dist_0', 'pop_dist_1',
       'uppop_dist_1', 'pop_dist_2', 'uppop_dist_2', 'pop_dist_3',
       'uppop_dist_3', 'pop_dist_4', 'uppop_dist_4','positive_word',"negative_word"]

In [47]:
clf = RandomForestClassifier(n_estimators=100)  
RandomForestModel= clf.fit(train_data[feature_columns],train_data["target"])
r = clf.score(test_data[feature_columns], test_data["target"])  
print("accuract",r)

accuract 0.7615900621118012


In [49]:
feature_importance=pd.DataFrame({"importance":clf.feature_importances_,"feature":train_data[feature_columns].columns})
print(feature_importance.sort_values(by="importance"))

            feature  importance
16          blog_tf    0.000053
17             ?_tf    0.000391
14       retweet_tf    0.000401
25        follow_tf    0.000416
19          post_tf    0.000618
27          vote_tf    0.000623
15          free_tf    0.000664
23        social_tf    0.000668
13         check_tf    0.000682
29         media_tf    0.000752
20           top_tf    0.000871
8        twitter_tf    0.000891
24            10_tf    0.000976
21        please_tf    0.001030
18          help_tf    0.001505
12         great_tf    0.001630
28        people_tf    0.002464
11           how_tf    0.002596
22           out_tf    0.002806
9           this_tf    0.004934
26           you_tf    0.005945
2       hashtag_num    0.008741
41    negative_word    0.010483
10            to_tf    0.010546
3          link_num    0.012267
4       mention_num    0.013717
40    positive_word    0.014167
1        word_count    0.040974
38       pop_dist_4    0.047571
37     uppop_dist_3    0.047652
35     u

In [51]:
len(feature_columns)

42